In [ ]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

In [ ]:
!pip install scikit-learn # Install the scikit-learn library

In [ ]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
import pandas as pd
df=pd.read_csv('p4dataset2020.txt',header=None, delim_whitespace=True)
df

<ipython-input-20-00be8049c074>:2: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df=pd.read_csv('p4dataset2020.txt',header=None, delim_whitespace=True)


FileNotFoundError: [Errno 2] No such file or directory: 'p4dataset2020.txt'

In [ ]:
gender=df[1]
population=df[2]
print(np.unique(population))
df.drop(df.columns[[0, 1, 2]],axis=1,inplace=True)
modes = np.array(df.mode().values[0,:])
gene_np=df.to_numpy()

In [ ]:
X_gene=binarized = np.where(gene_np!=modes, 1, 0)
y_gene=np.array(population)

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class GeneticDataset(Dataset):
    def __init__(self, sequences):
        self.sequences = sequences

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        seq = self.sequences[idx]
        # Convert sequence to numerical representation
        # For example, you could use one-hot encoding
        return torch.tensor(seq, dtype=torch.float32).unsqueeze(0)

# Prepare your data
sequences = gene_np # Your genetic sequences
dataset = GeneticDataset(sequences)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [ ]:
import torch.optim as optim

# Initialize the model
model = GeneticUNet()

# Define loss function and optimizer
criterion = nn.MSELoss()  # or another appropriate loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 100
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    for batch in dataloader:
        batch = batch.to(device)

        # Forward pass
        embeddings = model(batch)

        # Compute loss (you might need to adjust this based on your task)
        loss = criterion(embeddings, torch.zeros_like(embeddings))

        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# Save the trained model
torch.save(model.state_dict(), 'genetic_unet_model.pth')

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from sklearn.cluster import KMeans

class GeneticUNet(nn.Module):
    def __init__(self, in_channels=1, embedding_dim=3):
        super(GeneticUNet, self).__init__()

        # Encoder (Contracting Path)
        self.enc1 = self.conv_block(in_channels, 64)
        self.enc2 = self.conv_block(64, 128)
        self.enc3 = self.conv_block(128, 256)

        # Decoder (Expanding Path)
        self.dec3 = self.conv_block(256, 128)
        self.dec2 = self.conv_block(128, 64)
        self.dec1 = self.conv_block(64, embedding_dim)

        self.pool = nn.MaxPool1d(2)
        self.upsample = nn.Upsample(scale_factor=2, mode='nearest')

    def conv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv1d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm1d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv1d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm1d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        # Encoder
        e1 = self.enc1(x)
        e2 = self.enc2(self.pool(e1))
        e3 = self.enc3(self.pool(e2))

        # Decoder
        d3 = self.dec3(self.upsample(e3))
        d2 = self.dec2(self.upsample(d3))
        d1 = self.dec1(self.upsample(d2))

        # Global Average Pooling to get embedding
        embedding = F.adaptive_avg_pool1d(d1, 1).squeeze(-1)

        return embedding



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
X_gene.numpy()

In [ ]:
# Reshape your data
X_gene = X_gene.reshape(995, 1000)  # Assuming 995 is the number of samples

# Convert to PyTorch tensor and add channel dimension
X_gene = torch.tensor(X_gene, dtype=torch.float32).unsqueeze(1)  # Shape: [995, 1, 10101]

print("Input shape:", X_gene.shape)

model = GeneticUNet(in_channels=1, embedding_dim=64)

# Generate embeddings
model.eval()
with torch.no_grad():
    embeddings = model(X_gene)

print("Embeddings shape:", embeddings.shape)  # Should be [995, 64]

# Clustering
kmeans = KMeans(n_clusters=10, random_state=42)  # Adjust number of clusters as needed
cluster_labels = kmeans.fit_predict(embeddings.numpy())

print("Cluster labels shape:", cluster_labels.shape)  # Should be (995,)
print("First few cluster assignments:", cluster_labels[:10])  # Print first 10 cluster assignments

In [ ]:
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
emb=embeddings.numpy()
figure(figsize=(10, 10), dpi=80)
plt.scatter(emb[:,0],emb[:,1])

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ContrastiveLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, embeddings, labels):
        pairwise_distances = F.pairwise_distance(embeddings.unsqueeze(1), embeddings.unsqueeze(0))
        labels_matrix = labels.unsqueeze(0) == labels.unsqueeze(1)
        loss = torch.mean(labels_matrix.float() * pairwise_distances.pow(2) +
                          (1 - labels_matrix.float()) * F.relu(self.margin - pairwise_distances).pow(2))
        return loss

In [ ]:
le = LabelEncoder()
y_gene_encoded = le.fit_transform(y_gene)

# To get back original labels
# original_labels = le.inverse_transform(y_gene_encoded)

# Convert to PyTorch tensor
y_gene_tensor = torch.tensor(y_gene_encoded, dtype=torch.long)

In [ ]:
y_gene_tensor

In [ ]:
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

# Assuming X_gene is your input data and y_gene is your labels
X_gene = torch.tensor(X_gene, dtype=torch.float32).unsqueeze(1)  # Shape: [samples, 1, 10101]
le = LabelEncoder()
y_gene_encoded = le.fit_transform(y_gene)
y_gene_tensor = torch.tensor(y_gene_encoded, dtype=torch.long)  # Assuming y_gene is numeric. If not, encode it first.

# Create dataset and dataloader
dataset = TensorDataset(X_gene, y_gene)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Initialize the model
model = GeneticUNet(in_channels=1, embedding_dim=64)

# Define loss function and optimizer
criterion = ContrastiveLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 50
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch_X, batch_y in dataloader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)

        optimizer.zero_grad()
        embeddings = model(batch_X)
        loss = criterion(embeddings, batch_y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')

# Save the trained model
torch.save(model.state_dict(), 'genetic_unet_model.pth')

In [ ]:
!pip freeze